<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [3]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import plotly
import plotly.express as px

In [5]:
connection = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
query_3_1 = '''
SELECT
        COUNT(v.id)
FROM    
        public.vacancies v
'''

In [8]:
all_vac = pd.read_sql_query(query_3_1, connection)
print(all_vac)
print()
print(f'В базе данных предстаавлены {all_vac.iloc[0,0]} вакансий.')

   count
0  49197

В базе данных предстаавлены 49197 вакансий.


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [9]:
query_3_2 = '''
SELECT
        COUNT(e.id)
FROM    
        public.employers e
'''

In [11]:
all_emp = pd.read_sql_query(query_3_2, connection)
print(all_emp)
print()
print(f'В базе данных предстаавлен {all_emp.iloc[0,0]} работодатель.')

   count
0  23501

В базе данных предстаавлен 23501 работодатель.


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [12]:
query_3_3 = '''
SELECT
        COUNT(a.id)
FROM    
        public.areas a
'''

In [15]:
all_area = pd.read_sql_query(query_3_3, connection)
print(all_area)
print()
print(f'В базе данных предстаавлены {all_area.iloc[0,0]} региона.')

   count
0   1362

В базе данных предстаавлены 1362 региона.


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [17]:
query_3_4 = '''
SELECT
        COUNT(i.id)
FROM    
        public.industries i
'''

In [18]:
all_ind = pd.read_sql_query(query_3_4, connection)
print(all_ind)
print()
print(f'В базе данных предстаавлены {all_ind.iloc[0,0]} сферы деятельности работодателей.')

   count
0    294

В базе данных предстаавлены 294 сферы деятельности работодателей.


***

Предварительный анализ данных дает нам представление об общих количественных параметрах базы данных. А именно:
* общее количество вакансий 
* общее количество работодателей
* общее количество регионов размещения вакансий
* общее количество сфер деятельности работодателей

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [70]:
query_4_1 = ''' 
SELECT
        a.name name_area,
        COUNT(v.id)
FROM    
        public.vacancies v
        JOIN public.areas a ON v.area_id = a.id
GROUP BY 
        1 
ORDER BY 
        2 DESC
'''

In [71]:
vac_area = pd.read_sql_query(query_4_1, connection)
print(vac_area.head())


         name_area  count
0           Москва   5333
1  Санкт-Петербург   2851
2            Минск   2112
3      Новосибирск   2006
4           Алматы   1892


In [77]:
fig = px.treemap(
    data_frame=vac_area,
    path=['name_area'], 
    values='count', 
    height=500, 
    width=1000, 
    title='Количество вакансий в городах' 
)
fig.show()
fig.write_html('data/treemap_area_vacancies.html')

2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [22]:
query_4_2 = '''
SELECT
        COUNT(v.id)
FROM    
        public.vacancies v
WHERE 
        v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL
'''

In [72]:
salary_not_null = pd.read_sql_query(query_4_2, connection)
print(salary_not_null)


   count
0  24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [24]:
query_4_3 = '''
SELECT
        ROUND(AVG(v.salary_from))::INTEGER s_from,
        ROUND(AVG(v.salary_to))::INTEGER s_to
FROM    
        public.vacancies v
'''

In [73]:
from_to_salary = pd.read_sql_query(query_4_3, connection)
print(from_to_salary)


   s_from    s_to
0   71065  110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [74]:
query_4_4 = '''
SELECT DISTINCT
        v.schedule,
        v.employment,
        COUNT(v.id)
FROM    
        public.vacancies v
GROUP BY 
        1,2 
ORDER BY 
        3 DESC
'''

In [82]:
schedule_empl = pd.read_sql_query(query_4_4, connection)
print(schedule_empl.head())



           schedule           employment  count
0       Полный день     Полная занятость  35367
1  Удаленная работа     Полная занятость   7802
2     Гибкий график     Полная занятость   1593
3  Удаленная работа  Частичная занятость   1312
4    Сменный график     Полная занятость    940


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [28]:
query_4_5 = '''
SELECT 
        v.experience experience,
        COUNT(v.id) count
FROM    
        public.vacancies v
GROUP BY 
        1 
ORDER BY 
        2 
'''

In [84]:
vac_exp = pd.read_sql_query(query_4_5, connection)
print(vac_exp)


           experience  count
0         Более 6 лет   1337
1           Нет опыта   7197
2       От 3 до 6 лет  14511
3  От 1 года до 3 лет  26152


In [86]:
fig = px.pie(vac_exp, values='count', names='experience', title='Процентное соотношение количества вакансий в вариантах опыта работы ')
fig.show()
fig.write_html('data/pie_vac_eexp.html')

***

Детальный анализ вакансий  позволяет сформулировать следующие  выводы:
* В городах с большим населением - большое количество вакансий
* Лишь в половине представленных вакансий указана заработная плата (целесообразно рассмотреть этот признак в разрезе работодателей, регионов и сферы деятельности)
* Средняя зарплатная вилка по регионам 71000-110000 руб.(целесообразно сравнить с медианным и модальным значениями)
* Около 70% всех вакансий предполагают   тип занятости - полное трудоустройство, рабочий график - полный рабочий день
* Примерно в половине вакансий указан требуемый опыт работы от года до трех лет

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [30]:
query_5_1 = '''
WITH emp AS(SELECT 
                    e.name employer,
                    COUNT(v.id) vac_cnt
            FROM 
                    public.vacancies v
                    JOIN public.employers e ON v.employer_id = e.id
            GROUP BY 
                    e.id 
            ORDER BY 
                    2 DESC)
(SELECT 
        employer, vac_cnt
FROM 
        emp
LIMIT 
        1)
UNION 
(SELECT 
        employer, vac_cnt
FROM 
        emp
OFFSET 
        4
LIMIT 
        1)
ORDER BY 
        2 DESC
'''

In [31]:
empl_vac = pd.read_sql_query(query_5_1, connection)
print(empl_vac)
print()
print(f'Первое место по количеству вакансий: {empl_vac.iloc[0,0]}')
print(f'Пятое место по количеству вакансий: {empl_vac.iloc[1,0]}')

        employer  vac_cnt
0         Яндекс     1933
1  Газпром нефть      331

Первое место по количеству вакансий: Яндекс
Пятое место по количеству вакансий: Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [32]:
query_5_2 = '''
SELECT 
        a.name area, 
        count(v.id) vac_cnt,
        count(e.id) empl_cnt
FROM 
        public.areas a 
        left join public.vacancies v ON a.id = v.area_id
        left join public.employers e ON a.id = e.area
WHERE 
        v.id IS NULL
GROUP BY
        a.id  
ORDER BY 
        count(e.id) DESC
LIMIT 
        1
'''

In [33]:
area_no_vac = pd.read_sql(query_5_2, connection)
print(area_no_vac)
print()
print(f'Среди регионов, в которых нет вакансий, наибольшее количество работодателей в регионе {area_no_vac.iloc[0,0]}')

     area  vac_cnt  empl_cnt
0  Россия        0       410

Среди регионов, в которых нет вакансий, наибольшее количество работодателей в регионе Россия


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [34]:
query_5_3 = '''
SELECT 
        e.name employer,
        COUNT(DISTINCT a.id) area_cnt
FROM 
        public.vacancies v
        LEFT JOIN public.employers e ON e.id = v.employer_id
        LEFT JOIN public.areas a ON a.id = v.area_id 
GROUP BY 
        e.id
ORDER BY 
        COUNT(DISTINCT a.id) DESC
LIMIT 
        1
'''

In [35]:
area_count = pd.read_sql_query(query_5_3, connection)
print(area_count)
print()
print(f'Самая широкая география распространения вакансий у работодателя: {area_count.iloc[0,0]}')

  employer  area_cnt
0   Яндекс       181

Самая широкая география распространения вакансий у работодателя: Яндекс


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [88]:

query_5_4 = '''
SELECT 
        
        COUNT(DISTINCT e.id) emp_count
FROM 
        public.employers e
        LEFT JOIN public.employers_industries ei ON ei.employer_id = e.id
        
WHERE 
        ei.industry_id IS NULL
UNION
SELECT 
        
        COUNT(DISTINCT e.id) emp_count
FROM 
        public.employers e
        LEFT JOIN public.employers_industries ei ON ei.employer_id = e.id
        
WHERE 
        ei.industry_id IS NOT NULL
'''

In [89]:
count_emp = pd.read_sql_query(query_5_4, connection)
print(count_emp)
print()
print(f'Сфера деятельности не указана у {count_emp.iloc[0,0]} работодателей.')

   emp_count
0       8419
1      15082

Сфера деятельности не указана у 8419 работодателей.


In [90]:
industrie = ['не указана', 'указана']
count_emp['Industrie'] = industrie

fig = px.pie(count_emp, values='emp_count', names='Industrie', title='Соотношение количества работодателей с указанной  и неуказанной сферой деятельности')
fig.show()
fig.write_html('data/pie_cnt_emp.html')

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [39]:
query_5_5 = '''
SELECT 
        e.name employer
FROM 
        public.employers e
        LEFT JOIN public.employers_industries ei ON ei.employer_id = e.id
GROUP BY 
        e.id
HAVING 
        COUNT(DISTINCT ei.industry_id) = 4
ORDER BY 
        1
OFFSET 
        2 
LIMIT 
        1
'''

In [40]:
third_empl = pd.read_sql_query(query_5_5, connection)
print(third_empl)
print()
print(f'Компания {third_empl.iloc[0,0]} находится на третьем месте по алфавиту среди работодателей, у которых указано четыре сферы деятельности.')

  employer
0     2ГИС

Компания 2ГИС находится на третьем месте по алфавиту среди работодателей, у которых указано четыре сферы деятельности.


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [41]:
query_5_6 = '''
SELECT 
        COUNT(DISTINCT e.id)
FROM 
        public.employers e
        LEFT JOIN public.employers_industries ei ON ei.employer_id = e.id
        LEFT JOIN public.industries i ON ei.industry_id = i.id
WHERE 
        i.name = 'Разработка программного обеспечения'
'''

In [43]:
cnt_it_emp = pd.read_sql_query(query_5_6, connection)
print(cnt_it_emp)
print()
print(f'Разработкой програмного обеспечения занимаются {cnt_it_emp.iloc[0,0]} работодателя.')

   count
0   3553

Разработкой програмного обеспечения занимаются 3553 работодателя.


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [44]:
# получаем содержимое страницы Википедии в виде списка
list_from_wiki = pd.read_html('https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8')
city_df  = list_from_wiki[1] # второй элемент списка - датафрейм с городами-миллионниками
city_list = list(city_df['Город']) # получаем список городов
city_list.pop(-3) # удаляем 'Воронеж[]'
city_list.append('Воронеж') # добавляем 'Воронеж'
city_tuple = tuple(city_list) # преобразуем список в кортеж



In [45]:
query_5_7 = f'''
SELECT
        a.name city_name,
        COUNT(v.id) cnt
FROM 
        public.vacancies v 
        JOIN public.employers e ON e.id = v.employer_id
        JOIN public.areas a ON a.id = v.area_id
WHERE 
        e.name = 'Яндекс' AND 
        a.name IN {city_tuple}
GROUP BY
        a.id 
UNION 
SELECT
        'Total',
        COUNT(v.id)
FROM 
        public.vacancies v 
        JOIN public.employers e ON e.id = v.employer_id
        JOIN public.areas a ON a.id = v.area_id   
WHERE 
        e.name = 'Яндекс' AND 
        a.name IN {city_tuple}
ORDER BY 
        2 
'''

In [47]:
ya_df = pd.read_sql_query(query_5_7, connection)
print(ya_df)
print()
print(f'Вакансии компании ЯНДЕКС представлены во всех городах-миллионниках Росcии. Общее число вакансий в этих городах: {ya_df.iloc[-1,1]}')

          city_name  cnt
0              Омск   21
1         Челябинск   22
2        Красноярск   23
3         Волгоград   24
4             Пермь   25
5            Казань   25
6    Ростов-на-Дону   25
7               Уфа   26
8            Самара   26
9         Краснодар   30
10          Воронеж   32
11      Новосибирск   35
12  Нижний Новгород   36
13     Екатеринбург   39
14  Санкт-Петербург   42
15           Москва   54
16            Total  485

Вакансии компании ЯНДЕКС представлены во всех городах-миллионниках Росcии. Общее число вакансий в этих городах: 485


***

Выводы по анализу работодателей:
* Лидером по количеству вакансий и количеству регионов, где эти вакансии представлены, является компания Яндекс
* У трети работодателей не указана сфера деятельности
* Около 15% работодателей работают в области разработки програмного обеспечения

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [48]:
query_6_1 = '''
SELECT
        COUNT(v.id) cnt
FROM 
        public.vacancies v 
WHERE 
        LOWER(v.name) LIKE '%%data%%' OR LOWER(v.name) LIKE '%%данн%%'
'''

In [94]:
ds_vac_cnt = pd.read_sql_query(query_6_1, connection)
print(ds_vac_cnt)
print()
print(f'В базе данных представлена {ds_vac_cnt.iloc[0,0]} вакансия в сфере аналитики данных.')

    cnt
0  1771

В базе данных представлена 1771 вакансия в сфере аналитики данных.


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [50]:
query_6_2 = '''
SELECT
        COUNT(v.id) cnt
FROM 
        public.vacancies v 
WHERE 
        (v.name ILIKE  '%%data scientist%%' OR
        v.name ILIKE '%%data science%%' OR
        v.name ILIKE '%%исследователь данных%%' OR 
        v.name ILIKE '%%machine learning%%' OR
        v.name ILIKE '%%машинн%%обучен%%' OR 
        (v.name LIKE '%%ML%%' AND v.name NOT LIKE '%%HTML%%'))
        AND 
        (v.name ILIKE '%%junior%%' OR 
        v.experience = 'Нет опыта' OR 
        v.schedule = 'Стажировка')
'''

In [51]:
cnt_junior = pd.read_sql_query(query_6_2, connection)
print(cnt_junior)
print()
print(f'В базе данных представлена {cnt_junior.iloc[0,0]} вакансия для начинающего аналитика данных')

   cnt
0   51

В базе данных представлена 51 вакансия для начинающего аналитика данных


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [52]:
query_6_3 = '''
SELECT
        
        COUNT(v.id) cnt
FROM 
        public.vacancies v 
        
WHERE 
        (v.name ILIKE  '%%data scientist%%' OR
        v.name ILIKE '%%data science%%' OR
        v.name ILIKE '%%исследователь данных%%' OR 
        v.name ILIKE '%%machine learning%%' OR
        v.name ILIKE '%%машинн%%обучен%%' OR 
        (v.name LIKE '%%ML%%' AND v.name NOT LIKE '%%HTML%%'))
        AND 
        (v.key_skills ILIKE '%%SQL%%' OR 
        v.key_skills ILIKE '%%postgres%%')
'''

In [53]:
cnt_sql_postgres = pd.read_sql_query(query_6_3, connection)
print(cnt_sql_postgres)
print()
print(f'В базе данных представлена {cnt_sql_postgres.iloc[0,0]} вакансия, в которых в качестве ключевого навыка указаны SQL  и postgres. ')

   cnt
0  201

В базе данных представлена 201 вакансия, в которых в качестве ключевого навыка указаны SQL  и postgres. 


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [54]:
query_6_4 = '''
SELECT
        
        COUNT(v.id) cnt
FROM 
        public.vacancies v 
        
WHERE 
        (v.name ILIKE  '%%data scientist%%' OR
        v.name ILIKE '%%data science%%' OR
        v.name ILIKE '%%исследователь данных%%' OR 
        v.name ILIKE '%%machine learning%%' OR
        v.name ILIKE '%%машинн%%обучен%%' OR 
        (v.name LIKE '%%ML%%' AND v.name NOT LIKE '%%HTML%%'))
        AND 
        (v.key_skills ILIKE '%%Python%%' ) 
'''

In [55]:
cnt_python = pd.read_sql_query(query_6_4, connection)
print(cnt_python)
print()
print(f'В базе данных представлена {cnt_python.iloc[0,0]} вакансия, в которых в качестве ключевого навыка указан Python. ')

   cnt
0  351

В базе данных представлена 351 вакансия, в которых в качестве ключевого навыка указан Python. 


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [62]:
query_6_5 = '''
SELECT
        
        ROUND(AVG(LENGTH(v.key_skills) -                                --количество ключевых навыков определяется разностью длин строк  
        LENGTH(REPLACE(v.key_skills, CHR(9), '')) + 1), 2) mean_cnt     --с и без знаков-разделятелей соответственно
FROM 
        public.vacancies v 
        
WHERE 
        (v.name ILIKE  '%%data scientist%%' OR
        v.name ILIKE '%%data science%%' OR
        v.name ILIKE '%%исследователь данных%%' OR 
        v.name ILIKE '%%machine learning%%' OR
        v.name ILIKE '%%машинн%%обучен%%' OR 
        (v.name LIKE '%%ML%%' AND v.name NOT LIKE '%%HTML%%'))
        AND 
        (v.key_skills IS NOT NULL)

'''

In [63]:
mean_cnt_skill = pd.read_sql_query(query_6_5, connection)
print(mean_cnt_skill)
print()
print(f'Среднее количество ключевых навыков в вакансии аналитика данных: {mean_cnt_skill.iloc[0,0]} .')

   mean_cnt
0      6.41

Среднее количество ключевых навыков в вакансии аналитика данных: 6.41 .


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [64]:
query_6_6 = '''
SELECT
        v.experience,
        ROUND(AVG(COALESCE((v.salary_to + v.salary_from)/2, v.salary_to, v.salary_from)))::INTEGER average
FROM 
        public.vacancies v
WHERE 
        (v.name ILIKE  '%%data scientist%%' OR
        v.name ILIKE '%%data science%%' OR
        v.name ILIKE '%%исследователь данных%%' OR 
        v.name ILIKE '%%machine learning%%' OR
        v.name ILIKE '%%машинн%%обучен%%' OR 
        (v.name LIKE '%%ML%%' AND v.name NOT LIKE '%%HTML%%'))
        AND 
        (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
        AND 
        (v.experience IS NOT NULL)
GROUP BY 
        v.experience
'''

In [68]:
mean_salary_ds = pd.read_sql_query(query_6_6, connection)
print(mean_salary_ds)
print()
print(f'Дата-сайентист с опытом работы 3-6 лет может расчитывать на зарплату {mean_salary_ds.iloc[-1,1]} рублей.')


           experience  average
0           Нет опыта    74643
1  От 1 года до 3 лет   139675
2       От 3 до 6 лет   243115

Дата-сайентист с опытом работы 3-6 лет может расчитывать на зарплату 243115 рублей.


***

Выводы по предметному анализу:
* Вакансии в сфере DS  - 3 процента от общего количества вакансий
* Зарплата начинающего дата-сайентиста на уровне средней нижней границы зарплатной вилки 
* Количество вакансий для дата-сайентиста без опыта работы сравнительно не велико: лишь 3% от всех DS-вакансий
* Дата-сайентист с опытом работы от 3 до 6 лет может расчитывать на среднюю зарплату 240000 руб
* Зарплата дата-сайентиста с опытом работы свыше 6 лет обсуждается при собеседовании
* Знание языков Python  и PostgresSQL указано лишь в 10-15 процентах вакансий DS


# Общий вывод по проекту

Главный вывод по проекту: Data Scientist -  востребованная высокооплачиваемая должность, требующая владения большим количеством ключевых навыков. Вход в профессию для начинающего дата-сайентиста ограничен требованием работодателя к опыту работы.
Мне, как будующему соискателю, интересно отследить следующие показатели:
1. Распределение вакансий по работодателям.
2. Распределение вакансий по регионам.
3. Количество вакансий с типом трудоустройства - удаленная работа

In [95]:
query_1 = '''
SELECT
        e.name employer,
        COUNT(v.id) count
FROM 
        public.vacancies v
        LEFT JOIN public.employers e ON e.id = v.employer_id
WHERE   
        (v.name ILIKE  '%%data scientist%%' OR
        v.name ILIKE '%%data science%%' OR
        v.name ILIKE '%%исследователь данных%%' OR 
        v.name ILIKE '%%machine learning%%' OR
        v.name ILIKE '%%машинн%%обучен%%' OR 
        (v.name LIKE '%%ML%%' AND v.name NOT LIKE '%%HTML%%'))
GROUP BY 
        e.id
ORDER BY 
        2 DESC
LIMIT 
        5
'''
top_5 = pd.read_sql_query(query_1, connection)

fig = fig = px.bar(
    data_frame=top_5,
    x='employer',
    y='count',
    color='employer',
    title='ТОП-5 работодателей по количеству вакансий в DS'
)
fig.show()
fig.write_html('data/bar_top_5.html')

In [98]:
query_2 = '''
SELECT
        a.name name_area,
        COUNT(v.id) count
FROM 
        public.vacancies v
        LEFT JOIN public.areas a ON a.id = v.area_id
WHERE   
        (v.name ILIKE  '%%data scientist%%' OR
        v.name ILIKE '%%data science%%' OR
        v.name ILIKE '%%исследователь данных%%' OR 
        v.name ILIKE '%%machine learning%%' OR
        v.name ILIKE '%%машинн%%обучен%%' OR 
        (v.name LIKE '%%ML%%' AND v.name NOT LIKE '%%HTML%%'))
GROUP BY 
        a.id
ORDER BY 
        2 DESC
'''
area_ds  = pd.read_sql_query(query_2, connection)

fig = px.treemap(
    data_frame=area_ds,
    path=['name_area'], 
    values='count', 
    height=500, 
    width=1000, 
    title='Количество DS вакансий в регионах' 
)
fig.show()
fig.write_html('data/DS_vacancies.html')


In [6]:
query_3 = '''
SELECT
        COUNT(v.id)
FROM 
        public.vacancies v
WHERE   
        (v.name ILIKE  '%%data scientist%%' OR
        v.name ILIKE '%%data science%%' OR
        v.name ILIKE '%%исследователь данных%%' OR 
        v.name ILIKE '%%machine learning%%' OR
        v.name ILIKE '%%машинн%%обучен%%' OR 
        (v.name LIKE '%%ML%%' AND v.name NOT LIKE '%%HTML%%'))
        AND 
        v.schedule = 'Удалённая работа'
'''
remote_count = pd.read_sql_query(query_3, connection)
print(f'Удаленная работа предлагается в {remote_count.iloc[0,0]} вакансиях.')


Удаленная работа предлагается в 0 вакансиях.
